In [66]:
import os
from zipfile import ZipFile
import glob
import numpy as np
import xarray as xr
import rasterio
from scipy.stats import theilslopes

def download_data(variable, years, dir_prism):
    dir_ppt = os.path.join(dir_prism, variable+'/annual')
    os.makedirs(dir_ppt, exist_ok=True)
    for yr in years:
        url = 'https://services.nacse.org/prism/data/public/4km/' + variable + '/' + str(yr)
        ! wget --content-disposition {url} --directory-prefix {dir_ppt} -q

# unzip files 
def unzip_prism(dir_download):
    fn = glob.glob(dir_download + '/*.zip')
    for f in fn:
        dir_path, zip_file = os.path.split(f)
        filename_without_ext = os.path.splitext(zip_file)[0]

        out_dir = os.path.join(dir_path, filename_without_ext)
        os.makedirs(out_dir, exist_ok = True)

        with ZipFile(f, 'r') as zipfile:
            zipfile.extractall(out_dir)
            
# function used to calculate theilslpes
def theil_sen_trend(arr):
    x = np.arange(len(arr))
    # print(x)
    # print(arr)
    slopes = theilslopes(arr, x)[0]
    # slopes = np.mean(arr)
    # print(slopes)
    return slopes

    
def calc_trend_ca(dir_download, dir_trend):
    #  read all ppt data into combined ds
    fn = glob.glob(dir_download + '/**/*.bil')
    fn.sort()
    data = [xr.open_rasterio(f) for f in fn]
    combined_ds = xr.concat(data, dim = 'band')

    # interpolat the ds as there are missing data
    data_array = combined_ds
    data_array_interpolated = data_array.interpolate_na(dim='x', method='linear', use_coordinate='x')
    data_array_interpolated = data_array_interpolated.interpolate_na(dim='y', method='linear', use_coordinate='y')

    # calculate trend based on the interpolated data, this takes about 600s
    trend = xr.apply_ufunc(theil_sen_trend, data_array_interpolated, input_core_dims=[['band']], output_core_dims=[[]], vectorize=True)

     #plot California data
    trend_ca = trend.sel(y = slice(34,42), x = slice(-124, -117))
    trend_ca.plot()
    trend_ca.to_netcdf(dir_trend)


### Calculate trend for CA
* Total precipitation

In [5]:
dir_prism = '/Users/kehanyang/Library/CloudStorage/OneDrive-UW/research/pc3_meadow_ph/data/PRISM'
variable = 'ppt'
years = range(2001, 2023)
dir_download = os.path.join(dir_prism, variable+'/annual')

# donwload PRISM PPT data 
download_data(variable, years, dir_prism)
unzip_prism(dir_download)

dir_trend = './ca_ppt_trend_01_22.nc'
calc_trend_ca(dir_download = dir_download, dir_trend = dir_trend)

* Mean Temperature 

In [67]:
variable = 'tmean'
years = range(2001, 2023)
dir_download = os.path.join(dir_prism, variable+'/annual')

# donwload PRISM PPT data 
download_data(variable, years, dir_prism)
unzip_prism(dir_download)

dir_trend = './ca_tmean_trend_01_22.nc'
calc_trend_ca(dir_download = dir_download, dir_trend = dir_trend)

### Get the mean trend for each meadow extent

In [ ]:
!pip install rasterstats

In [ ]:
# read meadow shapfile 
import geopandas as gpd 
import rasterstats
shp = gpd.read_file('/Users/kehanyang/Documents/research/pc2_meadows/data/Sierra_meadows/SNMMPC_v2_one_acre_WGS.shp')
shp = gpd.read_file('/Users/kehanyang/Documents/research/pc2_meadows/data/Sierra_meadows/meadow_close_to_snotel/Meadow_close_to_SNOTEL.shp')

# read trend file using xarray
data_trend = xr.open_rasterio('./ca_ppt_trend_01_22.nc')

# calcumate the mean value for each polygon using rasterstats
stats = rasterstats.zonal.stats(shp, data_trend, stats = 'mean')

# Extract the mean values from the statistics.
mean_values = [stat['mean'] for stat in stats]

# Add the mean values as a new column to the GeoDataFrame.
shp['mean_trend_ppt'] = mean_values

# Print the GeoDataFrame with the mean values for each polygon.
print(gdf)